## Sentiwordnet mit Entfernen von Stopwords und Satzzeichen



In [10]:
import numpy as np # for array, linear algebra
import pandas as pd # for data processing
import matplotlib.pyplot as plt # to crreate stat and visualize data
import matplotlib as mpl  # to visualize data
import seaborn as sns # to visualize data
import matplotlib.colors as mcolors # to visualize colors
import string # collection of alphabets, words or other characters
import re # regular expression support
import nltk 
from nltk.corpus import stopwords
from string import punctuation 
# import gensim # representing documents as semantic vectors


from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/I570078/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/I570078/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/I570078/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/I570078/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Einstellung, um kompletten Inhalt der Spalte 'Text' anzuzeigen. 

In [11]:
pd.set_option('display.max_colwidth', None)

Einlesen der Daten

In [12]:
df_sentiwordnet_without_stopwords = pd.read_csv('../../Data-Preparation/traindata_without_stopwords_preprocessed.csv')

Überprüfung des Datensatzes

In [13]:
df_sentiwordnet_without_stopwords['target'].value_counts()

target
4    640000
0    640000
Name: count, dtype: int64

In [14]:
df_sentiwordnet_without_stopwords.head()

,Text,target,Ground_Truth_Label
0,Rise shining lol mins later planned rush door time,4,positive
1,2nd interview today looking promising,4,positive
2,feel pain,0,negative
3,4yr old son went exploring amp hiking today found Luv amphibians,4,positive
4,lonely housemates gone pub,0,negative


Dieser Classifier bildet den Durchschnitt aller Wortbedeutungen (Lemma), die ein jeweiliges Wort haben kann und achtet somit nicht auf den Kontext, sondern gibt eher einen allgemeinen Durchschnitt der Wortbedeutung als Sentiment-Score an.

In [15]:
# Check the types of the entries in the 'Text' column
df_sentiwordnet_without_stopwords['Text'].apply(type).value_counts()

Text
<class 'str'>      1273589
<class 'float'>       6411
Name: count, dtype: int64

In [18]:
df_sentiwordnet_without_stopwords['Text'] = df_sentiwordnet_without_stopwords['Text'].astype(str)


In [20]:
df_sentiwordnet_without_stopwords['Text'].apply(type).value_counts()

Text
<class 'str'>    1280000
Name: count, dtype: int64

In [16]:
def naiveSentiment(review):
    reviewPolarity = 0.0
    numExceptions = 0
    words = review.lower().split()
    
    for word in words:
        numMeanings = 0
        weight = 0.0
        synsets = list(swn.senti_synsets(word))
        if synsets:
            for meaning in synsets:
                pos_score = meaning.pos_score()
                neg_score = meaning.neg_score()
                if pos_score > neg_score:
                    weight += (pos_score - neg_score)
                    numMeanings += 1
                elif pos_score < neg_score:
                    weight -= (neg_score - pos_score)
                    numMeanings += 1
        else:
            numExceptions += 1
        if numMeanings > 0:
            reviewPolarity += (weight / numMeanings)
    return reviewPolarity

In [17]:
df_sentiwordnet_without_stopwords['Sentiwordnet_Classification'] = df_sentiwordnet_without_stopwords['Text'].apply(lambda x: 'positive' if naiveSentiment(x) >= 0 else 'negative')


AttributeError: 'float' object has no attribute 'lower'

In [ ]:
df_sentiwordnet_without_stopwords

,target,Text,Ground_Truth_Label,Sentiwordnet_Classification
0,0,Awww bummer shoulda got David Carr Third Day,negative,negative
1,0,upset update Facebook texting might cry result School today also Blah,negative,negative
2,0,dived many times ball Managed save 50 rest go bounds,negative,positive
3,0,whole body feels itchy like fire,negative,negative
4,0,behaving mad see,negative,positive
...,...,...,...,...
1599995,4,woke school best feeling ever,positive,positive
1599996,4,TheWDB com cool hear old Walt interviews ♫,positive,positive
1599997,4,ready MoJo Makeover Ask details,positive,positive
1599998,4,Happy 38th Birthday boo alll time Tupac Amaru Shakur,positive,positive


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_sentiwordnet_without_stopwords['Ground_Truth_Label'], df_sentiwordnet_without_stopwords['Sentiwordnet_Classification'])
print(f'Accuracy: {accuracy}')

Accuracy: 0.608200625
